In [1]:
import pandas as pd

In [7]:
sales1 = pd.read_csv("../Course_Materials_Part2/Video_Lecture_NBs/sales.csv", index_col= "Unnamed: 0")

In [8]:
sales1.head()

,Mon,Tue,Wed,Thu,Fri
Steven,34,27,15,NaN,33
Mike,45,9,74,87.0,12
Andi,17,33,54,8.0,29
Paul,87,67,27,45.0,7


In [9]:
sales2 = sales1.copy()

In [10]:
sales2.head()

,Mon,Tue,Wed,Thu,Fri
Steven,34,27,15,NaN,33
Mike,45,9,74,87.0,12
Andi,17,33,54,8.0,29
Paul,87,67,27,45.0,7


In [11]:
sales2.iloc[0,1] = 100
sales2.iloc[3,2] = 200

In [12]:
sales2.head()

,Mon,Tue,Wed,Thu,Fri
Steven,34,100,15,NaN,33
Mike,45,9,74,87.0,12
Andi,17,33,54,8.0,29
Paul,87,67,200,45.0,7


### Identify differences between tow datasets


#### Solution 1

In [14]:
sales2 == sales1

,Mon,Tue,Wed,Thu,Fri
Steven,True,False,True,False,True
Mike,True,True,True,True,True
Andi,True,True,True,True,True
Paul,True,True,False,True,True


#### Solution 2

In [15]:
sales1.where(~(sales1 == sales2))

,Mon,Tue,Wed,Thu,Fri
Steven,NaN,27.0,NaN,NaN,NaN
Mike,NaN,NaN,NaN,NaN,NaN
Andi,NaN,NaN,NaN,NaN,NaN
Paul,NaN,NaN,27.0,NaN,NaN


#### Solution 3

In [16]:
sales2.where(~(sales1 == sales2))

,Mon,Tue,Wed,Thu,Fri
Steven,NaN,100.0,NaN,NaN,NaN
Mike,NaN,NaN,NaN,NaN,NaN
Andi,NaN,NaN,NaN,NaN,NaN
Paul,NaN,NaN,200.0,NaN,NaN


#### Solution 4

In [17]:
sales_comp = pd.concat([sales1, sales2], axis=1, keys=["Day1", "Day2"])

In [18]:
sales_comp

Day1                   Day2                    
        Mon Tue Wed   Thu Fri  Mon  Tue  Wed   Thu Fri
Steven   34  27  15   NaN  33   34  100   15   NaN  33
Mike     45   9  74  87.0  12   45    9   74  87.0  12
Andi     17  33  54   8.0  29   17   33   54   8.0  29
Paul     87  67  27  45.0   7   87   67  200  45.0   7

#### Solution 5 
##### Creating a function

In [21]:
import numpy as np

In [24]:
def highlight_diff(data, color="yellow"):
    attr = f'background-color: {color}'
    other = data.xs('Day1', axis='columns', level=-2)
    return pd.DataFrame(np.where(data.ne(other, level=1), attr, ''),
                       index=data.index, columns=data.columns)

# using function
sales_comp.style.apply(highlight_diff, axis=None)

In [25]:
def diff_pd(df1, df2):
    """Identify differences between two pandas DataFrames"""
    assert (df1.columns == df2.columns).all(), \
        "DataFrame column names are different"
    if any(df1.dtypes != df2.dtypes):
        "Data Types are different, trying to convert"
        df2 = df2.astype(df1.dtypes)
    if df1.equals(df2):
        return None
    else:
        # need to account for np.nan != np.nan returning True
        diff_mask = (df1 != df2) & ~(df1.isnull() & df2.isnull())
        ne_stacked = diff_mask.stack()
        changed = ne_stacked[ne_stacked]
        changed.index.names = ['id', 'col']
        difference_locations = np.where(diff_mask)
        changed_from = df1.values[difference_locations]
        changed_to = df2.values[difference_locations]
        return pd.DataFrame({'from': changed_from, 'to': changed_to},
                            index=changed.index)

In [26]:
diff_pd(sales1, sales2)

,,from,to
id,col,,
Steven,Tue,27.0,100.0
Paul,Wed,27.0,200.0
